In [ ]:
from urllib.parse import urlencode, quote_plus
from urllib.request import urlopen, Request
import pandas as pd
import json, xmltodict
import pandas_profiling

service_key = 'VtS0z8ILedUMwjLe4kgmh6T8MWir/F9BsncSDqxFPsMsJaeTDkw3fDhNisjV5GdZerEeaEhxwDYbDhRG2SDz1g=='

In [ ]:
def get_gongdong_price(pnu, stdrYear, numOfRows, pageNo):    
    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : service_key, quote_plus('pnu') : pnu, quote_plus('stdrYear') : stdrYear, quote_plus('format') : 'json', quote_plus('numOfRows') : numOfRows, quote_plus('pageNo') : pageNo })
    
    url = 'http://apis.data.go.kr/1611000/nsdi/ApartHousingPriceService/attr/getApartHousingPriceAttr'
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read().decode('utf-8')
    data = json.loads(response_body)
    
    return data

In [ ]:
def get_price_yeonrib(LAWD_CD, DEAL_YMD):
    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : service_key, quote_plus('LAWD_CD') : LAWD_CD, quote_plus('DEAL_YMD') : DEAL_YMD })

    url_yeonrib = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcRHTrade'
    request = Request(url_yeonrib + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read().decode('utf-8')
    data = json.loads(json.dumps(xmltodict.parse(response_body)))
    
    return data

In [ ]:
code_df = pd.read_csv('/Users/jungwookim/dev/RealEstateInSeoul/beob_code.csv', encoding = "utf-8", index_col='법정동코드')
code_dic = code_df.loc[:,['법정동명']].to_dict()['법정동명']

seoul_code_list = list()
for code in code_dic:
    if code < 2000000000:
        seoul_code_10[code] = code_dic[code]
        
for key in seoul_code_list:
    seoul_code_list.append((str(key)[:5], str(key)[5:], seoul_code_10[key]))

In [ ]:
province_list = list()
for item in seoul_code_list:
    if item[0] not in province_list:
        province_list.append(item[0])

In [ ]:
def get_address(code_list):
    address_list = list()
    for item in code_list:
        # ~구라고 되어있는 곳까지 짜르고
        if item[2].find('구') == -1:
            continue
        else:
            split_point = item[2].find('구')
            tp = (item[0], item[2][:split_point+1], item[1], item[2][split_point+1:].replace(" ", ""))
            address_list.append(tp)
    return address_list

In [ ]:
# 예외 처리 방법
def get_jibunCode(jibun):
    if '-' in jibun:
        split_point = jibun.find('-')
        if len(jibun[:split_point]) == 1:
            if len(jibun[split_point+1:]) == 1:
                return '000'+jibun[:split_point]+'000'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 2:
                return '000'+jibun[:split_point]+'00'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 3:
                return '000'+jibun[:split_point]+'0'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 4:
                return '000'+jibun[:split_point]+jibun[split_point+1:]
        elif len(jibun[:split_point]) == 2:
            if len(jibun[split_point+1:]) == 1:
                return '00'+jibun[:split_point]+'000'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 2:
                return '00'+jibun[:split_point]+'00'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 3:
                return '00'+jibun[:split_point]+'0'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 4:
                return '00'+jibun[:split_point]+jibun[split_point+1:]
        elif len(jibun[:split_point]) == 3:
            if len(jibun[split_point+1:]) == 1:
                return '0'+jibun[:split_point]+'000'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 2:
                return '0'+jibun[:split_point]+'00'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 3:
                return '0'+jibun[:split_point]+'0'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 4:
                return '0'+jibun[:split_point]+jibun[split_point+1:]
        elif len(jibun[:split_point]) == 1:
            if len(jibun[split_point+1:]) == 4:
                return jibun[:split_point]+'000'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 2:
                return jibun[:split_point]+'00'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 3:
                return jibun[:split_point]+'0'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 4:
                return jibun[:split_point]+jibun[split_point+1:]
    else:
        if len(jibun) == 1:
            return '000'+jibun+'0000'
        elif len(jibun) == 2:
            return '00'+jibun+'0000'
        elif len(jibun) == 3:
            return '0'+jibun+'0000'
        elif len(jibun) == 4:
            return jibun+'0000'

In [ ]:
data2 = get_price_yeonrib('11110', '201712')
df2 = pd.DataFrame(data2['response']['body']['items']['item'])
df2.head()

# create yeonrib.csv

I'm going to create yeonrib.csv file in seoul at 2017

In [ ]:
with open('/Users/jungwookim/dev/RealEstateInSeoul/yeonrib.csv', 'a') as f:
    df_temp = pd.DataFrame(columns=df2.columns)
    df_temp.to_csv('yeonrib.csv', index=False)

    date_set = [str(x) for x in range(201701, 201713)]
    
    for date in date_set:
        for item in province_list:
            try:
                # 여기서 코드는 구까지 있는 5-digits 코드이다.
                data = get_price_yeonrib(item, date)
                dataFrame = pd.DataFrame(data['response']['body']['items']['item'])
                dataFrame.to_csv(f, header=False, columns=dataFrame.columns)
            except:
                continue

In [ ]:
df2 = pd.read_csv('yeonrib.csv')

# make pnu column for mapping in the future
I need to make pnu columns based on 지역코드, 법정동, 지번.
pnu consists of 19 digits which is like 1111010100100520018.

Let's decompose that into 5+5+1+4+4 digits.

* For the first 5 digits indicate 지역코드(서울시 종로구): 11110
* next 5 digtis indicate 법정동(청운동): 10100
* next 1 digit indicate register code: 1 (I wouldn't take anything except 1)
* next 4 digits indicate 지번: 0052 of 52-18
* last 4 digits indicate 지번: 0018 of 52-18

In [ ]:
# Add dongCode

address_list = get_address(seoul_code_list)
dong_list = list()
n = 0

for m, row in enumerate(df2.itertuples(index=True)):
    for item in address_list:
        if str(getattr(row, "지역코드")) in item:
            if getattr(row, "법정동") in item:
                dong_list.append(item[2])
                n += 1
    if len(dong_list) != (m + 1):
        dong_list.append(item[2])
        n += 1

In [ ]:
# add new columns
df2['dongCode'] = dong_list

# int to str
df2['지역코드'] = df2['지역코드'].apply(str)

There are some noise from data so I need to remove some NAN data.

In [ ]:
def remove_nan_rows(df):
    remove_index = list()
    for n, item in enumerate(pd.isnull(df['지번']).values):
        if item:
            remove_index.append(n)

    print(remove_index, "의 인덱스 rows가 삭제되었습니다.")
    df.drop(df.index[remove_index], inplace=True)

In [ ]:
remove_nan_rows(df2)

In [ ]:
# Add jibunCode
df2['jibunCode'] = df2['지번'].map(get_jibunCode)

In [ ]:
# create pnu
df2['pnu'] = df2['지역코드'] + df2['dongCode'] + '1' + df2['jibunCode']

# create gongdong.csv

Now, I'm going to create gongdong.csv file. It is much easier.

In [ ]:
df = get_gongdong_price('1111010100100040007', '2017', '10', '1')
df = pd.DataFrame(df['apartHousingPrices']['field'])
df.head()

In [ ]:
with open('/Users/jungwookim/dev/RealEstateInSeoul/gongdong.csv', 'a') as f:
    df_temp = pd.DataFrame(columns=df.columns)
    df_temp.to_csv('gongdong.csv', index=False)
    
    for code in seoul_code_10:
        for page in range(1, 3):
            try:
                data = get_gongdong_price(str(code), '2017', '100', str(page))
                data_frame = pd.DataFrame(data['apartHousingPrices']['field'])
                data_frame.to_csv(f, header=False, index_label = data_frame.columns)
                print(code, page)
            except:
                print("{} on {} PAGE NOT FOUND".format(code, page))
                continue

In [ ]:
pd.read_csv('gongdong.csv').head()

We're finally done with preparing data for analysis.